In [123]:
from bs4 import BeautifulSoup
from datetime import time
from datetime import datetime
from datetime import timedelta
from collections import Counter
import json
import requests
import csv
import pandas as pd
import numpy as np
import aiohttp
import asyncio
import concurrent.futures
import io
import re

In [124]:
# def pull_all_data(division):
#     base_url = 'https://c3po.crossfit.com/api/competitions/v2/competitions/open/2024/leaderboards'
#     main_query = f'view=0&division={division}&region=0&scaled=0&sort=0'
#     r = requests.get(f'{base_url}?{main_query}')
#     data = r.json()
#     page_count = data['pagination']['totalPages']
#     with open('NewMaleOpenLeaderboard.csv', 'w', newline='') as csvfile:
#         csv_writer = csv.writer(csvfile)
#         csv_writer.writerow(['CompetitorID', 'CompetitorName', 'CountryName', 'RegionName', 'AffiliateName', 'Gender', 'Height', 'Age', 'Weight', 'Overall Rank', '24.1 Rank', '24.2 Rank', '24.3 Rank'])
#         for i in range(1, page_count+1):
#             full_url = f'{base_url}?{main_query}&page={i}'
#             r = requests.get(full_url)
#             data = r.json()
#             for row in data['leaderboardRows']:
#                 row_out = [
#                     row['entrant']['competitorId'],
#                     row['entrant']['competitorName'],
#                     row['entrant']['countryOfOriginName'],
#                     row['entrant']['regionName'],
#                     row['entrant']['affiliateName'],
#                     row['entrant']['gender'],
#                     row['entrant']['height'],
#                     row['entrant']['age'],
#                     row['entrant']['weight'],
#                     row['overallRank'],
#                     row['scores'][0]['rank'],
#                     row['scores'][1]['rank'],
#                     row['scores'][2]['rank'],
#                 ]
#                 csv_writer.writerow(row_out)

# if __name__ == '__main__':
#     pull_all_data(1)

In [125]:
# def pull_all_data(division):
#     base_url = 'https://c3po.crossfit.com/api/competitions/v2/competitions/open/2024/leaderboards'
#     main_query = f'view=0&division={division}&region=0&scaled=0&sort=0'
#     r = requests.get(f'{base_url}?{main_query}')
#     data = r.json()
#     page_count = data['pagination']['totalPages']
#     with open('NewFemaleOpenLeaderboard.csv', 'w', newline='') as csvfile:
#         csv_writer = csv.writer(csvfile)
#         csv_writer.writerow(['CompetitorID', 'CompetitorName', 'CountryName', 'RegionName', 'AffiliateName', 'Gender', 'Height', 'Age', 'Weight', 'Overall Rank', '24.1 Rank', '24.2 Rank', '24.3 Rank'])
#         for i in range(1, page_count+1):
#             full_url = f'{base_url}?{main_query}&page={i}'
#             r = requests.get(full_url)
#             data = r.json()
#             for row in data['leaderboardRows']:
#                 row_out = [
#                     row['entrant']['competitorId'],
#                     row['entrant']['competitorName'],
#                     row['entrant']['countryOfOriginName'],
#                     row['entrant']['regionName'],
#                     row['entrant']['affiliateName'],
#                     row['entrant']['gender'],
#                     row['entrant']['height'],
#                     row['entrant']['age'],
#                     row['entrant']['weight'],
#                     row['overallRank'],
#                     row['scores'][0]['rank'],
#                     row['scores'][1]['rank'],
#                     row['scores'][2]['rank'],
#                 ]
#                 csv_writer.writerow(row_out)

# if __name__ == '__main__':
#     pull_all_data(2)

In [126]:
Men_Leaderboard = pd.read_csv('/Users/cdmoseley/dai_files/Capstone/Galvanize_Capstone_Crossfit/Data/MenOpenLeaderboard.csv')
Men_Leaderboard.head()

,CompetitorID,CompetitorName,CountryName,RegionName,AffiliateName,Gender,Height,Age,Weight,Overall Rank,24.1 Rank,24.2 Rank,24.3 Rank
0,173837,Jonne Koski,Finland,Europe,CrossFit 10K,M,172 cm,29,190 lb,1,27,2,1
1,310970,Saxon Panchik,United States,North America East,CrossFit Nolensville,M,69 in,28,185 lb,2,25,6,8
2,156156,Jay Crouch,Australia,Oceania,NaN,M,175 cm,25,88 kg,3,13,26,25
3,1408660,Luka Vunjak,Serbia,Europe,CrossFit Minus,M,173 cm,24,85 kg,4,22,55,4
4,2725,Noah Ohlsen,United States,North America East,Peak 360 CrossFit,M,67 in,33,190 lb,5,17,55,10


In [127]:
Women_Leaderboard = pd.read_csv('/Users/cdmoseley/dai_files/Capstone/Galvanize_Capstone_Crossfit/Data/WomenOpenLeaderboard.csv')
Women_Leaderboard.head()

,CompetitorID,CompetitorName,CountryName,RegionName,AffiliateName,Gender,Height,Age,Weight,Overall Rank,24.1 Rank,24.2 Rank,24.3 Rank
0,2080362,Mirjam von Rohr,Switzerland,Europe,CrossFit Ouf,F,NaN,22,NaN,1,1,37,2
1,1976672,Grace Walton,Australia,Oceania,NaN,F,164 cm,23,70 kg,2,8,3,33
2,1239579,Anikha Greer,Canada,North America East,Peak 360 CrossFit,F,154 cm,21,NaN,3,42,27,3
3,132258,Arielle Loewen,United States,North America West,NaN,F,63 in,30,150 lb,4,21,4,52
4,239148,Carolyne Prevost,Canada,North America East,CrossFit Colosseum,F,63 in,34,145 lb,5,15,59,17


In [128]:
# # Function to scrape Benchmark tables from Women athlete page
# def scrape_tables(athlete_id):
#     url = f"https://games.crossfit.com/athlete/{athlete_id}"
#     try:
#         response = requests.get(url)
#         response.raise_for_status()  # Raise an exception for 4xx or 5xx status codes
#         soup = BeautifulSoup(response.content, 'html.parser')
#         tables = soup.find_all('table')
#         if tables:
#             last_four_tables = tables[-4:]
#             athlete_data = {}
#             for table in last_four_tables:
#                 table_data = {}
#                 for row in table.find_all('tr'):
#                     cells = [cell.get_text(strip=True) for cell in row.find_all(['th', 'td'])]
#                     table_data[cells[0]] = cells[1]
#                 athlete_data.update(table_data)
#             return athlete_data
#         else:
#             print(f"No tables found for athlete ID {athlete_id}")
#             return None
#     except Exception as e:
#         print(f"Failed to retrieve webpage for athlete ID {athlete_id}: {e}")
#         return None

# # Load the DataFrame with athlete IDs
# athlete_ids = Women['CompetitorID'].tolist()

# # Sort athlete IDs numerically
# athlete_ids.sort(key=int)

# # Open a CSV file in write mode
# with open('WomenBenchmarks.csv', 'w', newline='') as csvfile:
#     # Create a CSV writer object
#     csv_writer = csv.writer(csvfile)
    
#     # Write headers
#     headers = ["Athlete ID", "Back Squat", "Chad1000x", "Clean and Jerk", "Deadlift", 
#                "Fight Gone Bad", "Filthy 50", "Fran", "Grace", "Helen", 
#                "L1 Benchmark", "Max Pull-Ups", "Run 5k", "Snatch", "Sprint 400m"]
#     csv_writer.writerow(headers)
    
#     # Set maximum number of concurrent workers
#     max_workers = 5  # Adjusted for Macbook Air
    
#     # Define a function to process each athlete ID
#     def process_athlete(athlete_id):
#         athlete_data = scrape_tables(athlete_id)
#         if athlete_data:
#             row_data = [athlete_id]
#             for header in headers[1:]:
#                 row_data.append(athlete_data.get(header, "--"))
#             csv_writer.writerow(row_data)
    
#     # Use ThreadPoolExecutor to execute scraping function concurrently
#     with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
#         executor.map(process_athlete, athlete_ids)

# print("Data has been written to WomenBenchmarks.csv successfully.")

In [129]:
# # Function to scrape Benchmark tables from Men athlete page
# def scrape_tables(athlete_id):
#     url = f"https://games.crossfit.com/athlete/{athlete_id}"
#     try:
#         response = requests.get(url)
#         response.raise_for_status()  # Raise an exception for 4xx or 5xx status codes
#         soup = BeautifulSoup(response.content, 'html.parser')
#         tables = soup.find_all('table')
#         if tables:
#             last_four_tables = tables[-4:]
#             athlete_data = {}
#             for table in last_four_tables:
#                 table_data = {}
#                 for row in table.find_all('tr'):
#                     cells = [cell.get_text(strip=True) for cell in row.find_all(['th', 'td'])]
#                     table_data[cells[0]] = cells[1]
#                 athlete_data.update(table_data)
#             return athlete_data
#         else:
#             print(f"No tables found for athlete ID {athlete_id}")
#             return None
#     except Exception as e:
#         print(f"Failed to retrieve webpage for athlete ID {athlete_id}: {e}")
#         return None

# # Load the DataFrame with athlete IDs
# athlete_ids = Men_Leaderboard['CompetitorID'].tolist()

# # Sort athlete IDs numerically
# athlete_ids.sort(key=int)

# # Open a CSV file in write mode
# with open('MenBenchmarks.csv', 'w', newline='') as csvfile:
#     # Create a CSV writer object
#     csv_writer = csv.writer(csvfile)
    
#     # Write headers
#     headers = ["Athlete ID", "Back Squat", "Chad1000x", "Clean and Jerk", "Deadlift", 
#                "Fight Gone Bad", "Filthy 50", "Fran", "Grace", "Helen", 
#                "L1 Benchmark", "Max Pull-Ups", "Run 5k", "Snatch", "Sprint 400m"]
#     csv_writer.writerow(headers)
    
#     # Set maximum number of concurrent workers
#     max_workers = 5  # Adjusted for Macbook Air
    
#     # Define a function to process each athlete ID
#     def process_athlete(athlete_id):
#         athlete_data = scrape_tables(athlete_id)
#         if athlete_data:
#             row_data = [athlete_id]
#             for header in headers[1:]:
#                 row_data.append(athlete_data.get(header, "--"))
#             csv_writer.writerow(row_data)
    
#     # Use ThreadPoolExecutor to execute scraping function concurrently
#     with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
#         executor.map(process_athlete, athlete_ids)

# print("Data has been written to MenBenchmarks.csv successfully.")

In [130]:
# # Function to Scrape Men's Profiles for past Open Results 
# def scrape_first_table(athlete_id):
#     url = f"https://games.crossfit.com/athlete/{athlete_id}"
#     try:
#         response = requests.get(url)
#         response.raise_for_status()  # Raise an exception for 4xx or 5xx status codes
#         soup = BeautifulSoup(response.content, 'html.parser')
#         table = soup.find('table')
#         if table:
#             table_data = {}
#             for row in table.find_all('tr'):
#                 cells = [cell.get_text(strip=True) for cell in row.find_all(['th', 'td'])]
#                 if len(cells) >= 2:
#                     year = cells[0]
#                     rank = cells[1]
#                     # Ensure year is between 2024 and 2011
#                     if year.isdigit() and 2024 >= int(year) >= 2011:
#                         table_data[year] = rank
#             return table_data
#         else:
#             print(f"No table found for athlete ID {athlete_id}")
#             return None
#     except Exception as e:
#         print(f"Failed to retrieve webpage for athlete ID {athlete_id}: {e}")
#         return None

In [131]:
# # Define max_workers globally
# max_workers = 5

# # Function to scrape the first table on the page
# def scrape_first_table(athlete_id):
#     url = f"https://games.crossfit.com/athlete/{athlete_id}"
#     try:
#         response = requests.get(url)
#         response.raise_for_status()  # Raise an exception for 4xx or 5xx status codes
#         soup = BeautifulSoup(response.content, 'html.parser')
#         table = soup.find('table')
#         if table:
#             table_data = {}
#             for row in table.find_all('tr'):
#                 cells = [cell.get_text(strip=True) for cell in row.find_all(['th', 'td'])]
#                 if len(cells) >= 2:
#                     year = cells[0]
#                     rank = cells[1]
#                     # Ensure year is between 2024 and 2011
#                     if year.isdigit() and 2024 >= int(year) >= 2011:
#                         table_data[year] = rank
#             return table_data
#         else:
#             print(f"No table found for athlete ID {athlete_id}")
#             return None
#     except Exception as e:
#         print(f"Failed to retrieve webpage for athlete ID {athlete_id}: {e}")
#         return None

# # Load the DataFrame with athlete IDs
# # Assuming you have a DataFrame named Men_Leaderboard and a column 'CompetitorID'
# # Replace this part with your actual data source
# athlete_ids = Men_Leaderboard['CompetitorID'].tolist()

# # Sort athlete IDs numerically
# athlete_ids.sort(key=int)

# # Open a CSV file in write mode for the first table
# with open('MenPastOpen.csv', 'w', newline='') as csvfile:
#     # Create a CSV writer object
#     csv_writer_first_table = csv.writer(csvfile)
    
#     # Write headers
#     headers_first_table = ["Athlete ID"] + [str(year) for year in range(2024, 2010, -1)]
#     csv_writer_first_table.writerow(headers_first_table)
    
#     # Define a function to process each athlete ID for the first table
#     def process_first_table(athlete_id):
#         athlete_data = scrape_first_table(athlete_id)
#         if athlete_data:
#             row_data = [athlete_id]
#             for year in range(2024, 2010, -1):
#                 row_data.append(athlete_data.get(str(year), "--"))
#             csv_writer_first_table.writerow(row_data)
    
#     # Use ThreadPoolExecutor to execute scraping function concurrently for the first table
#     with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
#         executor.map(process_first_table, athlete_ids)

# print("Data from the first table has been written to MenFirstTable.csv successfully.")

In [132]:
# # Define max_workers globally
# max_workers = 5

# # Function to scrape the first table on the page
# def scrape_first_table(athlete_id):
#     url = f"https://games.crossfit.com/athlete/{athlete_id}"
#     try:
#         response = requests.get(url)
#         response.raise_for_status()  # Raise an exception for 4xx or 5xx status codes
#         soup = BeautifulSoup(response.content, 'html.parser')
#         table = soup.find('table')
#         if table:
#             table_data = {}
#             for row in table.find_all('tr'):
#                 cells = [cell.get_text(strip=True) for cell in row.find_all(['th', 'td'])]
#                 if len(cells) >= 2:
#                     year = cells[0]
#                     rank = cells[1]
#                     # Ensure year is between 2024 and 2011
#                     if year.isdigit() and 2024 >= int(year) >= 2011:
#                         table_data[year] = rank
#             return table_data
#         else:
#             print(f"No table found for athlete ID {athlete_id}")
#             return None
#     except Exception as e:
#         print(f"Failed to retrieve webpage for athlete ID {athlete_id}: {e}")
#         return None

# # Load the DataFrame with athlete IDs
# # Assuming you have a DataFrame named Men_Leaderboard and a column 'CompetitorID'
# # Replace this part with your actual data source
# athlete_ids = Women_Leaderboard['CompetitorID'].tolist()

# # Sort athlete IDs numerically
# athlete_ids.sort(key=int)

# # Open a CSV file in write mode for the first table
# with open('WomenPastOpen.csv', 'w', newline='') as csvfile:
#     # Create a CSV writer object
#     csv_writer_first_table = csv.writer(csvfile)
    
#     # Write headers
#     headers_first_table = ["Athlete ID"] + [str(year) for year in range(2024, 2010, -1)]
#     csv_writer_first_table.writerow(headers_first_table)
    
#     # Define a function to process each athlete ID for the first table
#     def process_first_table(athlete_id):
#         athlete_data = scrape_first_table(athlete_id)
#         if athlete_data:
#             row_data = [athlete_id]
#             for year in range(2024, 2010, -1):
#                 row_data.append(athlete_data.get(str(year), "--"))
#             csv_writer_first_table.writerow(row_data)
    
#     # Use ThreadPoolExecutor to execute scraping function concurrently for the first table
#     with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
#         executor.map(process_first_table, athlete_ids)

# print("Data from the first table has been written to WomenFirstTable.csv successfully.")

In [133]:
# Specify the correct encoding
encoding = 'latin1'  # or 'ISO-8859-1', etc. - choose the one that works for your file

# Function to remove specific lines from a file
def remove_lines(file_path, lines_to_remove):
    with open(file_path, 'r', encoding=encoding) as file:
        lines = file.readlines()
    
    # Remove the specified lines
    for line_number in sorted(lines_to_remove, reverse=True):
        del lines[line_number]

    with open(file_path, 'w', encoding=encoding) as file:
        file.writelines(lines)

# List of line numbers to remove
lines_to_remove = [6335, 19690, 96172, 97074, 102157, 119426]  # Adjust line numbers as needed

# Remove specified lines from the CSV file
remove_lines('/Users/cdmoseley/dai_files/Capstone/Galvanize_Capstone_Crossfit/Data/WomenBenchmarks.csv', lines_to_remove)

# Read the modified CSV file into a DataFrame
Women_Benchmarks = pd.read_csv('/Users/cdmoseley/dai_files/Capstone/Galvanize_Capstone_Crossfit/Data/WomenBenchmarks.csv', encoding=encoding)

# Set pandas display options to show all columns without truncation
pd.set_option('display.max_columns', None)

# Display the DataFrame
Women_Benchmarks.head()

,Athlete ID,Back Squat,Chad1000x,Clean and Jerk,Deadlift,Fight Gone Bad,Filthy 50,Fran,Grace,Helen,L1 Benchmark,Max Pull-Ups,Run 5k,Snatch,Sprint 400m
0,1755,245 lb,--,165 lb,265 lb,293,--,4:29,2:25,11:09,--,--,25:08,120 lb,1:20
1,1775,303 lb,--,231 lb,345 lb,--,--,2:37,1:54,10:22,--,--,--,190 lb,--
2,1659,300 lb,--,235 lb,350 lb,--,--,2:15,--,--,--,--,23:51,190 lb,1:00
3,1802,250 lb,--,200 lb,330 lb,320,--,3:30,--,--,--,--,--,160 lb,1:23
4,1751,120 kg,--,97 kg,166 kg,--,22:54,3:39,1:52,9:11,--,--,--,73 kg,--


In [134]:
Men_Leaderboard = pd.read_csv('/Users/cdmoseley/dai_files/Capstone/Galvanize_Capstone_Crossfit/Data/MenOpenLeaderboard.csv')
Men_Leaderboard.head()

,CompetitorID,CompetitorName,CountryName,RegionName,AffiliateName,Gender,Height,Age,Weight,Overall Rank,24.1 Rank,24.2 Rank,24.3 Rank
0,173837,Jonne Koski,Finland,Europe,CrossFit 10K,M,172 cm,29,190 lb,1,27,2,1
1,310970,Saxon Panchik,United States,North America East,CrossFit Nolensville,M,69 in,28,185 lb,2,25,6,8
2,156156,Jay Crouch,Australia,Oceania,NaN,M,175 cm,25,88 kg,3,13,26,25
3,1408660,Luka Vunjak,Serbia,Europe,CrossFit Minus,M,173 cm,24,85 kg,4,22,55,4
4,2725,Noah Ohlsen,United States,North America East,Peak 360 CrossFit,M,67 in,33,190 lb,5,17,55,10


In [135]:
# Function to convert kilograms to pounds and round to the nearest pound
def kg_to_lb(weight):
    if isinstance(weight, str):
        if 'kg' in weight:
            return round(float(weight.split()[0]) * 2.20462)
        else:
            return round(float(weight.split()[0]))
    elif isinstance(weight, float) and np.isnan(weight):
        return np.nan  # Return NaN if the weight is NaN
    else:
        return round(weight * 2.20462)  # If the weight is already in lbs, just return it

# Apply the conversion function to the Weight column
Men_Leaderboard['Weight'] = Men_Leaderboard['Weight'].apply(kg_to_lb)

# Print the updated DataFrame
Men_Leaderboard.head()

,CompetitorID,CompetitorName,CountryName,RegionName,AffiliateName,Gender,Height,Age,Weight,Overall Rank,24.1 Rank,24.2 Rank,24.3 Rank
0,173837,Jonne Koski,Finland,Europe,CrossFit 10K,M,172 cm,29,190.0,1,27,2,1
1,310970,Saxon Panchik,United States,North America East,CrossFit Nolensville,M,69 in,28,185.0,2,25,6,8
2,156156,Jay Crouch,Australia,Oceania,NaN,M,175 cm,25,194.0,3,13,26,25
3,1408660,Luka Vunjak,Serbia,Europe,CrossFit Minus,M,173 cm,24,187.0,4,22,55,4
4,2725,Noah Ohlsen,United States,North America East,Peak 360 CrossFit,M,67 in,33,190.0,5,17,55,10


In [136]:
# Rename the 'Weight' column to 'Weight (lbs)'
Men_Leaderboard.rename(columns={'Weight': 'Weight (lbs)'}, inplace=True)

Men_Leaderboard.head()

,CompetitorID,CompetitorName,CountryName,RegionName,AffiliateName,Gender,Height,Age,Weight (lbs),Overall Rank,24.1 Rank,24.2 Rank,24.3 Rank
0,173837,Jonne Koski,Finland,Europe,CrossFit 10K,M,172 cm,29,190.0,1,27,2,1
1,310970,Saxon Panchik,United States,North America East,CrossFit Nolensville,M,69 in,28,185.0,2,25,6,8
2,156156,Jay Crouch,Australia,Oceania,NaN,M,175 cm,25,194.0,3,13,26,25
3,1408660,Luka Vunjak,Serbia,Europe,CrossFit Minus,M,173 cm,24,187.0,4,22,55,4
4,2725,Noah Ohlsen,United States,North America East,Peak 360 CrossFit,M,67 in,33,190.0,5,17,55,10


In [137]:
# Function to convert centimeters to inches
def cm_to_inches(height):
    if isinstance(height, str):
        if 'cm' in height:
            return round(float(height.split()[0]) * 0.393701)
        else:
            return int(height.split()[0])
    elif isinstance(height, float) and np.isnan(height):
        return np.nan  # Return NaN if the height is NaN
    else:
        return height  # If the height is already in inches, just return it

# Apply the conversion function to the Height column
Men_Leaderboard['Height'] = Men_Leaderboard['Height'].apply(cm_to_inches)

# Print the updated DataFrame
Men_Leaderboard.head()

,CompetitorID,CompetitorName,CountryName,RegionName,AffiliateName,Gender,Height,Age,Weight (lbs),Overall Rank,24.1 Rank,24.2 Rank,24.3 Rank
0,173837,Jonne Koski,Finland,Europe,CrossFit 10K,M,68.0,29,190.0,1,27,2,1
1,310970,Saxon Panchik,United States,North America East,CrossFit Nolensville,M,69.0,28,185.0,2,25,6,8
2,156156,Jay Crouch,Australia,Oceania,NaN,M,69.0,25,194.0,3,13,26,25
3,1408660,Luka Vunjak,Serbia,Europe,CrossFit Minus,M,68.0,24,187.0,4,22,55,4
4,2725,Noah Ohlsen,United States,North America East,Peak 360 CrossFit,M,67.0,33,190.0,5,17,55,10


In [138]:
# Rename the 'Height' column to 'Height (in))'
Men_Leaderboard.rename(columns={'Height': 'Height (in)'}, inplace=True)
Men_Leaderboard.head()

,CompetitorID,CompetitorName,CountryName,RegionName,AffiliateName,Gender,Height (in),Age,Weight (lbs),Overall Rank,24.1 Rank,24.2 Rank,24.3 Rank
0,173837,Jonne Koski,Finland,Europe,CrossFit 10K,M,68.0,29,190.0,1,27,2,1
1,310970,Saxon Panchik,United States,North America East,CrossFit Nolensville,M,69.0,28,185.0,2,25,6,8
2,156156,Jay Crouch,Australia,Oceania,NaN,M,69.0,25,194.0,3,13,26,25
3,1408660,Luka Vunjak,Serbia,Europe,CrossFit Minus,M,68.0,24,187.0,4,22,55,4
4,2725,Noah Ohlsen,United States,North America East,Peak 360 CrossFit,M,67.0,33,190.0,5,17,55,10


In [139]:
# Get unique values and sort them from highest to lowest
New_Height_sorted_Men = sorted(Men_Leaderboard['Height (in)'].unique(), reverse=True)
print(New_Height_sorted_Men)

[nan, 25801.0, 7087.0, 2756.0, 2124.0, 2112.0, 2040.0, 1189.0, 1071.0, 1000.0, 840.0, 780.0, 756.0, 752.0, 738.0, 728.0, 724.0, 713.0, 709.0, 708.0, 705.0, 701.0, 697.0, 693.0, 689.0, 685.0, 681.0, 677.0, 669.0, 665.0, 654.0, 601.0, 511.0, 510.0, 509.0, 508.0, 506.0, 492.0, 466.0, 444.0, 420.0, 393.0, 324.0, 310.0, 201.0, 194.0, 192.0, 191.0, 190.0, 188.0, 187.0, 186.0, 185.0, 184.0, 183.0, 182.0, 181.0, 180.0, 179.0, 178.0, 177.0, 176.0, 175.0, 174.0, 173.0, 172.0, 171.0, 170.0, 169.0, 168.0, 167.0, 166.0, 165.0, 162.0, 161.0, 118.0, 117.0, 110.0, 102.0, 99.0, 98.0, 97.0, 94.0, 93.0, 92.0, 90.0, 87.0, 86.0, 85.0, 84.0, 83.0, 82.0, 81.0, 80.0, 79.0, 78.0, 77.0, 76.0, 75.0, 74.0, 73.0, 72.0, 71.0, 70.0, 69.0, 68.0, 67.0, 66.0, 65.0, 64.0, 63.0, 62.0, 61.0, 60.0, 59.0, 58.0, 57.0, 56.0, 55.0, 54.0, 53.0, 50.0, 49.0, 47.0, 45.0, 43.0, 40.0, 37.0, 35.0, 33.0, 32.0, 31.0, 30.0, 29.0, 28.0, 27.0, 26.0, 25.0, 22.0, 18.0, 17.0, 12.0, 9.0, 7.0, 6.0, 5.0, 4.0, 2.0, 1.0, 0.0]


In [140]:
# Specify the correct encoding
encoding = 'latin1'  # or 'ISO-8859-1', etc. - choose the one that works for your file

# Function to remove lines with more than 15 fields from a file
def remove_lines_with_extra_fields(file_path):
    with open(file_path, 'r', encoding=encoding) as file:
        lines = file.readlines()

    # Identify lines with more than 15 fields and remove them
    new_lines = [line for line in lines if len(line.split(',')) <= 15]

    with open(file_path, 'w', encoding=encoding) as file:
        file.writelines(new_lines)

# Remove lines with more than 15 fields from the CSV file
remove_lines_with_extra_fields('/Users/cdmoseley/dai_files/Capstone/Galvanize_Capstone_Crossfit/Data/MenPastOpen.csv')

# Read the modified CSV file into a DataFrame
Men_Past = pd.read_csv('/Users/cdmoseley/dai_files/Capstone/Galvanize_Capstone_Crossfit/Data/MenPastOpen.csv', encoding=encoding)

# Display the DataFrame
Men_Past.head()

,Athlete ID,2024,2023,2022,2021,2020,2019,2018,2017,2016,2015,2014,2013,2012,2011
0,10942,68985thMen,62537thMen,66thMen Lower Extremity,--,126461stMen,64200thMen,19449thMen,20303rdMen,8704thMen,4733rdMen,3491stMen,1372ndMen,3243rdMen,5051stMen
1,10926,35977thMen,30964thMen,29723rdMen,21170thMen,18926thMen,19944thMen,16438thMen,12456thMen,11723rdMen,10968thMen,9164thMen,4686thMen,3880thMen,--
2,10976,164180thMen,143566thMen,119744thMen,--,--,--,--,--,--,--,--,--,14806thMen,--
3,10977,165637thMen,157742ndMen,87420thMen,56772ndMen,50805thMen,43038thMen,24249thMen,84288thMen,Men,Men,48836thMen,407thMen,661stMen,--
4,10966,60027thMen,49886thMen,55120thMen,37826thMen,22196thMen,34774thMen,45660thMen,55386thMen,46349thMen,29146thMen,19472ndMen,12970thMen,5729thMen,1996thMen


In [141]:
# Replace '--' with NaN
Men_Past.replace('--', np.nan, inplace=True)

# Print the updated DataFrame
Men_Past.head()

,Athlete ID,2024,2023,2022,2021,2020,2019,2018,2017,2016,2015,2014,2013,2012,2011
0,10942,68985thMen,62537thMen,66thMen Lower Extremity,NaN,126461stMen,64200thMen,19449thMen,20303rdMen,8704thMen,4733rdMen,3491stMen,1372ndMen,3243rdMen,5051stMen
1,10926,35977thMen,30964thMen,29723rdMen,21170thMen,18926thMen,19944thMen,16438thMen,12456thMen,11723rdMen,10968thMen,9164thMen,4686thMen,3880thMen,NaN
2,10976,164180thMen,143566thMen,119744thMen,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14806thMen,NaN
3,10977,165637thMen,157742ndMen,87420thMen,56772ndMen,50805thMen,43038thMen,24249thMen,84288thMen,Men,Men,48836thMen,407thMen,661stMen,NaN
4,10966,60027thMen,49886thMen,55120thMen,37826thMen,22196thMen,34774thMen,45660thMen,55386thMen,46349thMen,29146thMen,19472ndMen,12970thMen,5729thMen,1996thMen


In [142]:
#This gets rid of all the letters 
def extract_numbers(string):
    if pd.isnull(string):  # If the value is NaN, return NaN
        return np.nan
    elif isinstance(string, float):  # If the value is float, return NaN
        return np.nan
    else:
        numbers = re.findall(r'\d+', string)  # Extract numbers using regex
        if numbers:  # If numbers are found
            return int(numbers[0])  # Convert the first number found to an integer
        else:  # If no numbers found
            return np.nan  # Return NaN

# Apply the function to all columns except 'Athlete ID'
Men_Past[Men_Past.columns.difference(['Athlete ID'])] = Men_Past[Men_Past.columns.difference(['Athlete ID'])].applymap(extract_numbers)

# Print the updated DataFrame
Men_Past.head()

,Athlete ID,2024,2023,2022,2021,2020,2019,2018,2017,2016,2015,2014,2013,2012,2011
0,10942,68985.0,62537.0,66.0,NaN,126461.0,64200.0,19449.0,20303.0,8704.0,4733.0,3491.0,1372.0,3243.0,5051.0
1,10926,35977.0,30964.0,29723.0,21170.0,18926.0,19944.0,16438.0,12456.0,11723.0,10968.0,9164.0,4686.0,3880.0,NaN
2,10976,164180.0,143566.0,119744.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14806.0,NaN
3,10977,165637.0,157742.0,87420.0,56772.0,50805.0,43038.0,24249.0,84288.0,NaN,NaN,48836.0,407.0,661.0,NaN
4,10966,60027.0,49886.0,55120.0,37826.0,22196.0,34774.0,45660.0,55386.0,46349.0,29146.0,19472.0,12970.0,5729.0,1996.0


In [143]:
# Define a function to add the word "Rank" after all the years in the columns
def add_rank_suffix(col):
    if col.isdigit():
        return col + ' Rank'
    else:
        return col

# Rename the columns except 'Athlete ID'
Men_Past.rename(columns={col: add_rank_suffix(col) for col in Men_Past.columns if col != 'Athlete ID'}, inplace=True)

Men_Past.head()

,Athlete ID,2024 Rank,2023 Rank,2022 Rank,2021 Rank,2020 Rank,2019 Rank,2018 Rank,2017 Rank,2016 Rank,2015 Rank,2014 Rank,2013 Rank,2012 Rank,2011 Rank
0,10942,68985.0,62537.0,66.0,NaN,126461.0,64200.0,19449.0,20303.0,8704.0,4733.0,3491.0,1372.0,3243.0,5051.0
1,10926,35977.0,30964.0,29723.0,21170.0,18926.0,19944.0,16438.0,12456.0,11723.0,10968.0,9164.0,4686.0,3880.0,NaN
2,10976,164180.0,143566.0,119744.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14806.0,NaN
3,10977,165637.0,157742.0,87420.0,56772.0,50805.0,43038.0,24249.0,84288.0,NaN,NaN,48836.0,407.0,661.0,NaN
4,10966,60027.0,49886.0,55120.0,37826.0,22196.0,34774.0,45660.0,55386.0,46349.0,29146.0,19472.0,12970.0,5729.0,1996.0


In [144]:
# Function to remove lines with more than 15 fields from a CSV file
def remove_lines_with_extra_fields(file_path):
    with open(file_path, 'r', encoding='latin1') as file:
        reader = csv.reader(file)
        lines = [line for line in reader if len(line) <= 15]

    with open(file_path, 'w', encoding='latin1') as file:
        writer = csv.writer(file)
        writer.writerows(lines)

# Remove lines with more than 15 fields from the CSV file
remove_lines_with_extra_fields('/Users/cdmoseley/dai_files/Capstone/Galvanize_Capstone_Crossfit/Data/MenBenchmarks.csv')

# Read the modified CSV file into a DataFrame
Men_Benchmarks = pd.read_csv('/Users/cdmoseley/dai_files/Capstone/Galvanize_Capstone_Crossfit/Data/MenBenchmarks.csv')

# Display the DataFrame
Men_Benchmarks.head()


,Athlete ID,Back Squat,Chad1000x,Clean and Jerk,Deadlift,Fight Gone Bad,Filthy 50,Fran,Grace,Helen,L1 Benchmark,Max Pull-Ups,Run 5k,Snatch,Sprint 400m
0,88,455 lb,--,315 lb,518 lb,407,18:23,2:20,1:49,7:28,--,--,21:34,265 lb,0:54
1,1624,135 kg,--,113 kg,160 kg,398,22:16,4:00,2:16,9:22,--,--,23:32,77 kg,--
2,1618,325 lb,--,245 lb,415 lb,--,--,3:46,2:57,--,--,--,19:22,165 lb,1:05
3,86,335 lb,--,265 lb,415 lb,393,21:27,3:46,2:54,8:16,--,--,22:05,210 lb,1:04
4,1633,330 lb,--,250 lb,375 lb,314,--,3:57,4:00,--,--,--,20:00,198 lb,1:10


In [145]:
del Men_Benchmarks['Max Pull-Ups']
del Men_Benchmarks['Chad1000x']

In [146]:
# Replace '--' with NaN
Men_Benchmarks.replace('--', np.nan, inplace=True)

# Print the updated DataFrame
Men_Benchmarks.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 176450 entries, 0 to 176449
Data columns (total 13 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   Athlete ID      176418 non-null  object
 1   Back Squat      54704 non-null   object
 2   Clean and Jerk  50119 non-null   object
 3   Deadlift        55998 non-null   object
 4   Fight Gone Bad  9822 non-null    object
 5   Filthy 50       5766 non-null    object
 6   Fran            24313 non-null   object
 7   Grace           18782 non-null   object
 8   Helen           14213 non-null   object
 9   L1 Benchmark    624 non-null     object
 10  Run 5k          23022 non-null   object
 11  Snatch          47019 non-null   object
 12  Sprint 400m     11331 non-null   object
dtypes: object(13)
memory usage: 17.5+ MB


In [147]:
# Define a function to remove 'lb' and convert to pounds
def clean_and_convert(weight):
    try:
        if 'kg' in str(weight):
            return pd.to_numeric(weight.split()[0]) * 2.20462
        elif 'lb' in str(weight):
            return pd.to_numeric(weight.split()[0])
        else:
            return None
    except Exception as e:
        print(f"Error occurred: {e}")
        return None

# Columns to clean and convert
columns_to_clean = ['Back Squat', 'Clean and Jerk', 'Deadlift', 'Snatch']

# Apply the cleaning and conversion function to the specified columns
Men_Benchmarks[columns_to_clean] = Men_Benchmarks[columns_to_clean].applymap(clean_and_convert)

Men_Benchmarks.head()

,Athlete ID,Back Squat,Clean and Jerk,Deadlift,Fight Gone Bad,Filthy 50,Fran,Grace,Helen,L1 Benchmark,Run 5k,Snatch,Sprint 400m
0,88,455.0000,315.00000,518.0000,407,18:23,2:20,1:49,7:28,NaN,21:34,265.00000,0:54
1,1624,297.6237,249.12206,352.7392,398,22:16,4:00,2:16,9:22,NaN,23:32,169.75574,NaN
2,1618,325.0000,245.00000,415.0000,NaN,NaN,3:46,2:57,NaN,NaN,19:22,165.00000,1:05
3,86,335.0000,265.00000,415.0000,393,21:27,3:46,2:54,8:16,NaN,22:05,210.00000,1:04
4,1633,330.0000,250.00000,375.0000,314,NaN,3:57,4:00,NaN,NaN,20:00,198.00000,1:10


In [148]:
# Rename the Columns
Men_Benchmarks.rename(columns={'Back Squat': 'Back Squat (lbs)', 'Clean and Jerk':'Clean and Jerk (lbs)', 'Deadlift': 'Deadlift (lbs)', 'Snatch': 'Snatch (lbs)'}, inplace=True)
Men_Benchmarks.head()

,Athlete ID,Back Squat (lbs),Clean and Jerk (lbs),Deadlift (lbs),Fight Gone Bad,Filthy 50,Fran,Grace,Helen,L1 Benchmark,Run 5k,Snatch (lbs),Sprint 400m
0,88,455.0000,315.00000,518.0000,407,18:23,2:20,1:49,7:28,NaN,21:34,265.00000,0:54
1,1624,297.6237,249.12206,352.7392,398,22:16,4:00,2:16,9:22,NaN,23:32,169.75574,NaN
2,1618,325.0000,245.00000,415.0000,NaN,NaN,3:46,2:57,NaN,NaN,19:22,165.00000,1:05
3,86,335.0000,265.00000,415.0000,393,21:27,3:46,2:54,8:16,NaN,22:05,210.00000,1:04
4,1633,330.0000,250.00000,375.0000,314,NaN,3:57,4:00,NaN,NaN,20:00,198.00000,1:10


START THE PROCESS TO JOIN HERE 

In [149]:
# Remove rows with duplicate Athlete IDs from Men_Past
Men_Past_no_duplicates = Men_Past.drop_duplicates(subset='Athlete ID')


In [150]:
# Remove rows with duplicate Athlete IDs from Men_Benchmarks
Men_Benchmarks_no_duplicates = Men_Benchmarks.drop_duplicates(subset='Athlete ID')

In [151]:
# Rename Column
Men_Leaderboard.rename(columns={'CompetitorID': 'Athlete ID'}, inplace=True)

In [152]:
# Remove rows with duplicate IDs from Men_Benchmarks
Men_Leaderboard_no_duplicates = Men_Leaderboard.drop_duplicates(subset='Athlete ID')

In [153]:
# Merge Men_Past with Men_Benchmarks
merged_df = pd.merge(Men_Past_no_duplicates, Men_Benchmarks_no_duplicates, on='Athlete ID', how='inner')
merged_df.head()

,Athlete ID,2024 Rank,2023 Rank,2022 Rank,2021 Rank,2020 Rank,2019 Rank,2018 Rank,2017 Rank,2016 Rank,2015 Rank,2014 Rank,2013 Rank,2012 Rank,2011 Rank,Back Squat (lbs),Clean and Jerk (lbs),Deadlift (lbs),Fight Gone Bad,Filthy 50,Fran,Grace,Helen,L1 Benchmark,Run 5k,Snatch (lbs),Sprint 400m
0,10942,68985.0,62537.0,66.0,NaN,126461.0,64200.0,19449.0,20303.0,8704.0,4733.0,3491.0,1372.0,3243.0,5051.0,425.0,285.0,585.0,NaN,NaN,2:32,1:47,7:13,NaN,NaN,225.0,NaN
1,10926,35977.0,30964.0,29723.0,21170.0,18926.0,19944.0,16438.0,12456.0,11723.0,10968.0,9164.0,4686.0,3880.0,NaN,365.0,275.0,405.0,429,19:47,3:16,3:13,8:16,NaN,19:38,190.0,1:00
2,10976,164180.0,143566.0,119744.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14806.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,10977,165637.0,157742.0,87420.0,56772.0,50805.0,43038.0,24249.0,84288.0,NaN,NaN,48836.0,407.0,661.0,NaN,400.0,300.0,485.0,NaN,NaN,2:00,NaN,NaN,NaN,NaN,255.0,NaN
4,10966,60027.0,49886.0,55120.0,37826.0,22196.0,34774.0,45660.0,55386.0,46349.0,29146.0,19472.0,12970.0,5729.0,1996.0,315.0,225.0,400.0,NaN,NaN,3:41,NaN,9:01,NaN,21:55,165.0,NaN


In [154]:
# Convert 'Athlete ID' to string (object) in Men_Leaderboard
Men_Leaderboard_no_duplicates['Athlete ID'] = Men_Leaderboard_no_duplicates['Athlete ID'].astype(str)

# Merge merged_df with Men_Leaderboard
PreClean_Crossfit = pd.merge(merged_df, Men_Leaderboard_no_duplicates, on='Athlete ID', how='left')

PreClean_Crossfit.head()

,Athlete ID,2024 Rank,2023 Rank,2022 Rank,2021 Rank,2020 Rank,2019 Rank,2018 Rank,2017 Rank,2016 Rank,2015 Rank,2014 Rank,2013 Rank,2012 Rank,2011 Rank,Back Squat (lbs),Clean and Jerk (lbs),Deadlift (lbs),Fight Gone Bad,Filthy 50,Fran,Grace,Helen,L1 Benchmark,Run 5k,Snatch (lbs),Sprint 400m,CompetitorName,CountryName,RegionName,AffiliateName,Gender,Height (in),Age,Weight (lbs),Overall Rank,24.1 Rank,24.2 Rank,24.3 Rank
0,10942,68985.0,62537.0,66.0,NaN,126461.0,64200.0,19449.0,20303.0,8704.0,4733.0,3491.0,1372.0,3243.0,5051.0,425.0,285.0,585.0,NaN,NaN,2:32,1:47,7:13,NaN,NaN,225.0,NaN,Everett Sloan,Canada,North America East,CrossFit Bytown,M,67.0,45.0,200.0,68985.0,51383.0,84516.0,80225.0
1,10926,35977.0,30964.0,29723.0,21170.0,18926.0,19944.0,16438.0,12456.0,11723.0,10968.0,9164.0,4686.0,3880.0,NaN,365.0,275.0,405.0,429,19:47,3:16,3:13,8:16,NaN,19:38,190.0,1:00,Kevin Wood,Canada,North America East,CrossFit Moncton,M,75.0,43.0,196.0,35977.0,74333.0,17860.0,27790.0
2,10976,164180.0,143566.0,119744.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14806.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,John Lazaro,United States,North America East,CrossFit Bridgewater,M,69.0,46.0,180.0,164180.0,156669.0,154294.0,142271.0
3,10977,165637.0,157742.0,87420.0,56772.0,50805.0,43038.0,24249.0,84288.0,NaN,NaN,48836.0,407.0,661.0,NaN,400.0,300.0,485.0,NaN,NaN,2:00,NaN,NaN,NaN,NaN,255.0,NaN,Matthew Haynes,United States,North America West,CrossFit Westwood,M,71.0,31.0,180.0,165637.0,160697.0,154294.0,142271.0
4,10966,60027.0,49886.0,55120.0,37826.0,22196.0,34774.0,45660.0,55386.0,46349.0,29146.0,19472.0,12970.0,5729.0,1996.0,315.0,225.0,400.0,NaN,NaN,3:41,NaN,9:01,NaN,21:55,165.0,NaN,Nick DiMatteo,United States,North America East,CrossFit OTG,M,68.0,48.0,165.0,60027.0,47496.0,89266.0,54110.0


In [155]:
# # Output Crossfit to a CSV file
# PreClean_Crossfit.to_csv('Pre-Clean_Men_Crossfit.csv', index=False)

In [156]:
Men_Clean_Me = pd.read_csv('/Users/cdmoseley/dai_files/Capstone/Galvanize_Capstone_Crossfit/Data/Pre-Clean_Men_Crossfit.csv')
Men_Clean_Me.head()

/var/folders/xm/kc32pwjj5fj5xqf43yb4dz340000gn/T/ipykernel_44180/401025724.py:1: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  Men_Clean_Me = pd.read_csv('/Users/cdmoseley/dai_files/Capstone/Galvanize_Capstone_Crossfit/Data/Pre-Clean_Men_Crossfit.csv')


,Athlete ID,2024 Rank,2023 Rank,2022 Rank,2021 Rank,2020 Rank,2019 Rank,2018 Rank,2017 Rank,2016 Rank,2015 Rank,2014 Rank,2013 Rank,2012 Rank,2011 Rank,Back Squat (lbs),Clean and Jerk (lbs),Deadlift (lbs),Fight Gone Bad,Filthy 50,Fran,Grace,Helen,L1 Benchmark,Run 5k,Snatch (lbs),Sprint 400m,CompetitorName,CountryName,RegionName,AffiliateName,Gender,Height (in),Age,Weight (lbs),Overall Rank,24.1 Rank,24.2 Rank,24.3 Rank
0,10942,68985.0,62537.0,66.0,NaN,126461.0,64200.0,19449.0,20303.0,8704.0,4733.0,3491.0,1372.0,3243.0,5051.0,425.0,285.0,585.0,NaN,NaN,2:32,1:47,7:13,NaN,NaN,225.0,NaN,Everett Sloan,Canada,North America East,CrossFit Bytown,M,67.0,45.0,200.0,68985.0,51383.0,84516.0,80225.0
1,10926,35977.0,30964.0,29723.0,21170.0,18926.0,19944.0,16438.0,12456.0,11723.0,10968.0,9164.0,4686.0,3880.0,NaN,365.0,275.0,405.0,429.0,19:47,3:16,3:13,8:16,NaN,19:38,190.0,1:00,Kevin Wood,Canada,North America East,CrossFit Moncton,M,75.0,43.0,196.0,35977.0,74333.0,17860.0,27790.0
2,10976,164180.0,143566.0,119744.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14806.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,John Lazaro,United States,North America East,CrossFit Bridgewater,M,69.0,46.0,180.0,164180.0,156669.0,154294.0,142271.0
3,10977,165637.0,157742.0,87420.0,56772.0,50805.0,43038.0,24249.0,84288.0,NaN,NaN,48836.0,407.0,661.0,NaN,400.0,300.0,485.0,NaN,NaN,2:00,NaN,NaN,NaN,NaN,255.0,NaN,Matthew Haynes,United States,North America West,CrossFit Westwood,M,71.0,31.0,180.0,165637.0,160697.0,154294.0,142271.0
4,10966,60027.0,49886.0,55120.0,37826.0,22196.0,34774.0,45660.0,55386.0,46349.0,29146.0,19472.0,12970.0,5729.0,1996.0,315.0,225.0,400.0,NaN,NaN,3:41,NaN,9:01,NaN,21:55,165.0,NaN,Nick DiMatteo,United States,North America East,CrossFit OTG,M,68.0,48.0,165.0,60027.0,47496.0,89266.0,54110.0


In [157]:
# Define the desired order of column names
desired_column_order = [
    'Athlete ID',
    'Gender',
    'CompetitorName',
    'CountryName',
    'RegionName',
    'AffiliateName',
    'Height (in)',
    'Age',
    'Weight (lbs)',
    'Overall Rank',
    '24.1 Rank',
    '24.2 Rank',
    '24.3 Rank',
    '2024 Rank',
    '2023 Rank',
    '2022 Rank',
    '2021 Rank',
    '2020 Rank',
    '2019 Rank',
    '2018 Rank',
    '2017 Rank',
    '2016 Rank',
    '2015 Rank',
    '2014 Rank',
    '2013 Rank',
    '2012 Rank',
    '2011 Rank',
    'Back Squat (lbs)',
    'Deadlift (lbs)',
    'Clean and Jerk (lbs)',
    'Snatch (lbs)',
    'Fight Gone Bad',
    'Filthy 50',
    'Fran',
    'Grace',
    'Helen',
    'L1 Benchmark',
    'Run 5k',
    'Sprint 400m'
 
]
Men_Clean_Me=Men_Clean_Me[desired_column_order]
Men_Clean_Me.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 176260 entries, 0 to 176259
Data columns (total 39 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   Athlete ID            176259 non-null  object 
 1   Gender                176255 non-null  object 
 2   CompetitorName        176254 non-null  object 
 3   CountryName           175672 non-null  object 
 4   RegionName            176255 non-null  object 
 5   AffiliateName         155236 non-null  object 
 6   Height (in)           90301 non-null   float64
 7   Age                   176255 non-null  float64
 8   Weight (lbs)          89489 non-null   float64
 9   Overall Rank          176255 non-null  float64
 10  24.1 Rank             176255 non-null  float64
 11  24.2 Rank             176255 non-null  float64
 12  24.3 Rank             176255 non-null  float64
 13  2024 Rank             176257 non-null  float64
 14  2023 Rank             96307 non-null   float64
 15  

In [158]:
# Rename the Column Names
Men_Clean_Me.rename(columns={'CompetitorName':'Competitor Name', 'CountryName':'County Name', 'RegionName':'Region Name', 'AffiliateName': 'Affiliate Name'}, inplace=True)
Men_Clean_Me.head()

,Athlete ID,Gender,Competitor Name,County Name,Region Name,Affiliate Name,Height (in),Age,Weight (lbs),Overall Rank,24.1 Rank,24.2 Rank,24.3 Rank,2024 Rank,2023 Rank,2022 Rank,2021 Rank,2020 Rank,2019 Rank,2018 Rank,2017 Rank,2016 Rank,2015 Rank,2014 Rank,2013 Rank,2012 Rank,2011 Rank,Back Squat (lbs),Deadlift (lbs),Clean and Jerk (lbs),Snatch (lbs),Fight Gone Bad,Filthy 50,Fran,Grace,Helen,L1 Benchmark,Run 5k,Sprint 400m
0,10942,M,Everett Sloan,Canada,North America East,CrossFit Bytown,67.0,45.0,200.0,68985.0,51383.0,84516.0,80225.0,68985.0,62537.0,66.0,NaN,126461.0,64200.0,19449.0,20303.0,8704.0,4733.0,3491.0,1372.0,3243.0,5051.0,425.0,585.0,285.0,225.0,NaN,NaN,2:32,1:47,7:13,NaN,NaN,NaN
1,10926,M,Kevin Wood,Canada,North America East,CrossFit Moncton,75.0,43.0,196.0,35977.0,74333.0,17860.0,27790.0,35977.0,30964.0,29723.0,21170.0,18926.0,19944.0,16438.0,12456.0,11723.0,10968.0,9164.0,4686.0,3880.0,NaN,365.0,405.0,275.0,190.0,429.0,19:47,3:16,3:13,8:16,NaN,19:38,1:00
2,10976,M,John Lazaro,United States,North America East,CrossFit Bridgewater,69.0,46.0,180.0,164180.0,156669.0,154294.0,142271.0,164180.0,143566.0,119744.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14806.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,10977,M,Matthew Haynes,United States,North America West,CrossFit Westwood,71.0,31.0,180.0,165637.0,160697.0,154294.0,142271.0,165637.0,157742.0,87420.0,56772.0,50805.0,43038.0,24249.0,84288.0,NaN,NaN,48836.0,407.0,661.0,NaN,400.0,485.0,300.0,255.0,NaN,NaN,2:00,NaN,NaN,NaN,NaN,NaN
4,10966,M,Nick DiMatteo,United States,North America East,CrossFit OTG,68.0,48.0,165.0,60027.0,47496.0,89266.0,54110.0,60027.0,49886.0,55120.0,37826.0,22196.0,34774.0,45660.0,55386.0,46349.0,29146.0,19472.0,12970.0,5729.0,1996.0,315.0,400.0,225.0,165.0,NaN,NaN,3:41,NaN,9:01,NaN,21:55,NaN


In [159]:
# Convert 'Fight Gone Bad' column to float
Men_Clean_Me['Fight Gone Bad'] = Men_Clean_Me['Fight Gone Bad'].astype(float)

In [160]:
Men_Clean_Me.head()

,Athlete ID,Gender,Competitor Name,County Name,Region Name,Affiliate Name,Height (in),Age,Weight (lbs),Overall Rank,24.1 Rank,24.2 Rank,24.3 Rank,2024 Rank,2023 Rank,2022 Rank,2021 Rank,2020 Rank,2019 Rank,2018 Rank,2017 Rank,2016 Rank,2015 Rank,2014 Rank,2013 Rank,2012 Rank,2011 Rank,Back Squat (lbs),Deadlift (lbs),Clean and Jerk (lbs),Snatch (lbs),Fight Gone Bad,Filthy 50,Fran,Grace,Helen,L1 Benchmark,Run 5k,Sprint 400m
0,10942,M,Everett Sloan,Canada,North America East,CrossFit Bytown,67.0,45.0,200.0,68985.0,51383.0,84516.0,80225.0,68985.0,62537.0,66.0,NaN,126461.0,64200.0,19449.0,20303.0,8704.0,4733.0,3491.0,1372.0,3243.0,5051.0,425.0,585.0,285.0,225.0,NaN,NaN,2:32,1:47,7:13,NaN,NaN,NaN
1,10926,M,Kevin Wood,Canada,North America East,CrossFit Moncton,75.0,43.0,196.0,35977.0,74333.0,17860.0,27790.0,35977.0,30964.0,29723.0,21170.0,18926.0,19944.0,16438.0,12456.0,11723.0,10968.0,9164.0,4686.0,3880.0,NaN,365.0,405.0,275.0,190.0,429.0,19:47,3:16,3:13,8:16,NaN,19:38,1:00
2,10976,M,John Lazaro,United States,North America East,CrossFit Bridgewater,69.0,46.0,180.0,164180.0,156669.0,154294.0,142271.0,164180.0,143566.0,119744.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14806.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,10977,M,Matthew Haynes,United States,North America West,CrossFit Westwood,71.0,31.0,180.0,165637.0,160697.0,154294.0,142271.0,165637.0,157742.0,87420.0,56772.0,50805.0,43038.0,24249.0,84288.0,NaN,NaN,48836.0,407.0,661.0,NaN,400.0,485.0,300.0,255.0,NaN,NaN,2:00,NaN,NaN,NaN,NaN,NaN
4,10966,M,Nick DiMatteo,United States,North America East,CrossFit OTG,68.0,48.0,165.0,60027.0,47496.0,89266.0,54110.0,60027.0,49886.0,55120.0,37826.0,22196.0,34774.0,45660.0,55386.0,46349.0,29146.0,19472.0,12970.0,5729.0,1996.0,315.0,400.0,225.0,165.0,NaN,NaN,3:41,NaN,9:01,NaN,21:55,NaN


Clean Each Column Individually 

In [161]:
Men_Clean_Me.columns

Index(['Athlete ID', 'Gender', 'Competitor Name', 'County Name', 'Region Name',
       'Affiliate Name', 'Height (in)', 'Age', 'Weight (lbs)', 'Overall Rank',
       '24.1 Rank', '24.2 Rank', '24.3 Rank', '2024 Rank', '2023 Rank',
       '2022 Rank', '2021 Rank', '2020 Rank', '2019 Rank', '2018 Rank',
       '2017 Rank', '2016 Rank', '2015 Rank', '2014 Rank', '2013 Rank',
       '2012 Rank', '2011 Rank', 'Back Squat (lbs)', 'Deadlift (lbs)',
       'Clean and Jerk (lbs)', 'Snatch (lbs)', 'Fight Gone Bad', 'Filthy 50',
       'Fran', 'Grace', 'Helen', 'L1 Benchmark', 'Run 5k', 'Sprint 400m'],
      dtype='object')

In [162]:
Men_Clean_Me['Athlete ID'].unique()

array(['10942', '10926', '10976', ..., 2729468, 2731196, 2729564],
      dtype=object)

In [163]:
Men_Clean_Me['Gender'].unique()

array(['M', nan, 'F'], dtype=object)

In [164]:
Men_Clean_Me['Competitor Name'].unique()

array(['Everett Sloan', 'Kevin Wood', 'John Lazaro', ..., 'JP Earnest',
       'David Crabb', 'Anthony Guarascio'], dtype=object)

In [165]:
Men_Clean_Me['County Name'].unique()

array(['Canada', 'United States', 'New Zealand', 'United Kingdom',
       'Germany', 'Kenya', 'Brazil', 'Australia', 'South Africa',
       'Denmark', 'Costa Rica', 'Sweden', 'Netherlands', 'Ireland',
       'France', 'Italy', 'Guatemala', 'Argentina', 'Lithuania',
       'Korea, Republic of', 'Philippines', 'Finland', 'India', 'Mexico',
       'Chile', 'Israel', 'Switzerland', 'Ecuador', 'Spain', 'Norway',
       'Hungary', 'Iceland', 'Czech Republic', 'Malaysia',
       'Brunei Darussalam', 'Colombia', 'Laos', 'Cayman Islands',
       'Portugal', 'United Arab Emirates', 'Belgium', 'Latvia', 'Peru',
       'China', 'Malta', 'Puerto Rico', nan, 'Bulgaria', 'Greece',
       'Japan', 'Singapore', 'Austria', 'Poland', 'Dominican Republic',
       'Croatia', 'Saudi Arabia', 'Slovakia', 'Zimbabwe', 'Barbados',
       'Hong Kong, China', 'Turkey', 'Estonia', 'Serbia', 'Bermuda',
       'Ukraine', 'Slovenia', 'Mauritius', 'Sri Lanka', 'Panama',
       'Namibia', 'Venezuela', 'Romania', 'Thail

In [166]:
Men_Clean_Me.rename(columns={'County Name':'Country Name'}, inplace=True)

In [167]:
Men_Clean_Me['Region Name'].unique()

array(['North America East', 'North America West', 'Oceania', 'Europe',
       'Africa', 'South America', 'Asia', nan], dtype=object)

In [168]:
Men_Clean_Me['Affiliate Name'].unique()

array(['CrossFit Bytown', 'CrossFit Moncton', 'CrossFit Bridgewater', ...,
       'CrossFit ARX South', 'CrossFit Gold Lions', 'CrossFit Kello'],
      dtype=object)

In [169]:
# Get unique values and sort them from highest to lowest
Height_sorted = sorted(Men_Clean_Me['Height (in)'].unique(), reverse=True)
print(Height_sorted)

[nan, 25801.0, 7087.0, 2756.0, 2124.0, 2112.0, 2040.0, 1189.0, 1071.0, 1000.0, 840.0, 780.0, 756.0, 752.0, 738.0, 728.0, 724.0, 713.0, 709.0, 708.0, 705.0, 701.0, 697.0, 693.0, 689.0, 685.0, 681.0, 677.0, 669.0, 665.0, 654.0, 601.0, 511.0, 510.0, 509.0, 508.0, 506.0, 492.0, 466.0, 444.0, 420.0, 393.0, 324.0, 310.0, 201.0, 194.0, 192.0, 191.0, 190.0, 188.0, 187.0, 186.0, 185.0, 184.0, 183.0, 182.0, 181.0, 180.0, 179.0, 178.0, 177.0, 176.0, 175.0, 174.0, 173.0, 172.0, 171.0, 170.0, 169.0, 168.0, 167.0, 166.0, 165.0, 162.0, 161.0, 118.0, 117.0, 110.0, 102.0, 99.0, 98.0, 97.0, 94.0, 93.0, 92.0, 90.0, 87.0, 86.0, 85.0, 84.0, 83.0, 82.0, 81.0, 80.0, 79.0, 78.0, 77.0, 76.0, 75.0, 74.0, 73.0, 72.0, 71.0, 70.0, 69.0, 68.0, 67.0, 66.0, 65.0, 64.0, 63.0, 62.0, 61.0, 60.0, 59.0, 58.0, 57.0, 56.0, 55.0, 54.0, 53.0, 50.0, 47.0, 45.0, 43.0, 40.0, 37.0, 35.0, 33.0, 32.0, 31.0, 30.0, 29.0, 28.0, 27.0, 26.0, 25.0, 22.0, 18.0, 17.0, 12.0, 9.0, 7.0, 6.0, 5.0, 4.0, 2.0, 1.0, 0.0]


In [170]:
# Count the occurrences of each unique value in the 'col1' column
value_counts = Men_Clean_Me['Height (in)'].value_counts()

# Sort the value counts in descending order
sorted_value_counts = value_counts.sort_index()

# Set display options to show all rows
pd.set_option('display.max_rows', None)

# Print the sorted counts
print(sorted_value_counts)

Height (in)
0.0            4
1.0           66
2.0            9
4.0            1
5.0           57
6.0          169
7.0            1
9.0            1
12.0           1
17.0           1
18.0           1
22.0           2
25.0           2
26.0           6
27.0           5
28.0           9
29.0           3
30.0           5
31.0           4
32.0           2
33.0           4
35.0           2
37.0           2
40.0           1
43.0           1
45.0           1
47.0           2
50.0           3
53.0           3
54.0           2
55.0           5
56.0           2
57.0           1
58.0           6
59.0           9
60.0          89
61.0          57
62.0          83
63.0         308
64.0         661
65.0        1807
66.0        3199
67.0        7415
68.0        8064
69.0       12550
70.0       14023
71.0       11530
72.0       12087
73.0        6659
74.0        5458
75.0        2743
76.0        1764
77.0         623
78.0         303
79.0         126
80.0          61
81.0          31
82.0           2
83

In [171]:
# Replace values less than 48 or greater than 84 with NaN
Men_Clean_Me.loc[(Men_Clean_Me['Height (in)'] < 48) | (Men_Clean_Me['Height (in)'] > 84), 'Height (in)'] = pd.NaT

In [172]:
Men_Clean_Me['Age'].unique()

array([45., 43., 46., 31., 48., 50., 40., 42., 39., 36., 38., 37., 53.,
       47., 33., 35., 51., 52., 34., 44., 41., 32., 54., 49., 30., 29.,
       28., 27., nan, 26., 25., 24., 18., 23., 16., 22., 19., 20., 21.,
       17.])

In [173]:
# Replace values less than 50 or greater than 550 with NaN
Men_Clean_Me.loc[(Men_Clean_Me['Weight (lbs)'] < 50) | (Men_Clean_Me['Weight (lbs)'] > 550), 'Weight (lbs)'] = pd.NaT

In [174]:
# Count the occurrences of each unique value in the 'col1' column
weight_value_counts = Men_Clean_Me['Weight (lbs)'].value_counts()

# Sort the value counts in descending order
sorted_value_counts = weight_value_counts.sort_index()

# Set display options to show all rows
pd.set_option('display.max_rows', None)

# Print the sorted counts
print(sorted_value_counts)

Weight (lbs)
54.0        1
55.0        1
56.0        3
60.0        4
61.0        2
62.0        2
63.0        3
64.0        6
65.0        5
66.0        3
67.0       10
68.0        8
69.0        8
70.0       19
71.0       15
72.0       23
73.0       26
74.0       16
75.0       35
76.0       24
77.0       18
78.0       43
79.0       18
80.0       51
81.0       19
82.0       33
83.0       25
84.0       30
85.0       22
86.0       12
87.0       16
88.0       15
89.0        7
90.0       27
91.0       12
92.0       15
93.0       11
94.0        6
95.0       25
96.0        3
97.0        2
98.0        9
99.0        3
100.0       4
101.0       1
102.0       3
103.0       2
104.0       1
105.0       3
106.0       4
108.0       4
110.0      13
112.0       4
113.0       1
114.0       1
115.0      10
116.0       1
117.0       5
118.0       5
119.0      10
120.0      14
121.0      20
122.0       4
123.0      17
124.0       4
125.0      28
126.0      23
127.0       3
128.0      43
129.0       6
130.0  

In [175]:
Men_Clean_Me['Overall Rank'].unique()

array([ 68985.,  35977., 164180., ..., 126635., 160643., 163043.])

In [176]:
Men_Clean_Me['24.3 Rank'].unique()

array([ 80225.,  27790., 142271., ..., 130751., 122355., 120224.])

In [177]:
# Replace values less than 45 or greater than 600 with NaN
Men_Clean_Me.loc[(Men_Clean_Me['Back Squat (lbs)'] < 45) | (Men_Clean_Me['Back Squat (lbs)'] > 600), 'Back Squat (lbs)'] = pd.NaT

In [178]:
# Count the occurrences of each unique value in the 'col1' column
BS_value_counts = Men_Clean_Me['Back Squat (lbs)'].value_counts()

# Sort the value counts in descending order
BS_sorted_value_counts = BS_value_counts.sort_index()

# Set display options to show all rows
pd.set_option('display.max_rows', None)

# Print the sorted counts
print(BS_sorted_value_counts)

Back Squat (lbs)
45.0                     2
50.0                     1
50.70625999999999        1
55.1155                  2
56.0                     1
59.52474                 2
60.0                     1
66.1386                  3
69.0                     2
70.0                     1
75.0                     3
77.0                     1
77.1617                  1
78.0                     1
80.0                     1
83.0                     1
85.0                     1
85.98017999999999        1
87.0                     1
88.1848                  5
90.0                     5
90.38942                 1
93.0                     1
94.79866                 1
95.0                     9
98.0                     1
99.0                     2
99.2079                 10
100.0                    9
101.0                    1
101.41252                1
105.0                    3
105.82175999999998       1
106.0                    1
110.0                    9
110.231                 19
113.0      

In [179]:
# Replace values less than 44 or greater than 700 with NaN
Men_Clean_Me.loc[(Men_Clean_Me['Deadlift (lbs)'] < 44) | (Men_Clean_Me['Deadlift (lbs)'] > 700), 'Deadlift (lbs)'] = pd.NaT

In [180]:
# Count the occurrences of each unique value in the 'col1' column
DL_value_counts = Men_Clean_Me['Deadlift (lbs)'].value_counts()

# Sort the value counts in descending order
DL_sorted_value_counts = DL_value_counts.sort_index()

# Set display options to show all rows
pd.set_option('display.max_rows', None)

# Print the sorted counts
print(DL_sorted_value_counts)

Deadlift (lbs)
44.0                     1
44.0924                  6
45.0                     2
46.29702                 1
50.0                     1
65.0                     1
69.0                     1
72.0                     1
75.0                     4
77.1617                  1
79.36631999999999        1
80.0                     2
83.0                     1
83.77556                 1
85.0                     2
88.1848                  1
90.0                     1
94.79866                 2
95.0                     3
99.0                     1
99.2079                  3
100.0                    4
105.0                    3
105.82175999999998       1
108.0                    2
110.0                    2
110.231                  4
112.43562                1
114.64024                1
115.0                    6
119.04948                1
120.0                    6
121.2541                 2
125.0                    6
125.66334                1
126.0                    1
130.0        

In [181]:
# Replace values less than 44 or greater than 500 with NaN
Men_Clean_Me.loc[(Men_Clean_Me['Clean and Jerk (lbs)'] < 44) | (Men_Clean_Me['Clean and Jerk (lbs)'] > 500), 'Clean and Jerk (lbs)'] = pd.NaT

In [182]:
# Count the occurrences of each unique value in the 'col1' column
CJ_value_counts = Men_Clean_Me['Clean and Jerk (lbs)'].value_counts()

# Sort the value counts in descending order
CJ_sorted_value_counts = CJ_value_counts.sort_index()

# Set display options to show all rows
pd.set_option('display.max_rows', None)

# Print the sorted counts
print(CJ_sorted_value_counts)

Clean and Jerk (lbs)
44.0924                  5
45.0                    12
50.0                     2
51.0                     1
52.91087999999999        1
55.0                     4
55.1155                  3
56.0                     1
59.52474                 1
60.0                    10
61.72935999999999        1
62.0                     1
63.0                     1
64.0                     1
65.0                     5
66.1386                 12
67.0                     1
68.0                     2
69.0                     1
70.0                    10
71.0                     1
72.75246                 2
74.0                     1
74.95707999999999        1
75.0                    17
77.0                     2
77.1617                 18
78.0                     1
79.36631999999999        2
80.0                    13
81.57094                 2
83.0                     1
83.77556                 3
85.0                    16
86.0                     1
87.0                     1
88.0   

In [183]:
# Replace values less than 44 or greater than 450 with NaN
Men_Clean_Me.loc[(Men_Clean_Me['Snatch (lbs)'] < 44) | (Men_Clean_Me['Snatch (lbs)'] > 450), 'Snatch (lbs)'] = pd.NaT

In [184]:
# Count the occurrences of each unique value in the 'col1' column
SN_value_counts = Men_Clean_Me['Snatch (lbs)'].value_counts()

# Sort the value counts in descending order
SN_sorted_value_counts = SN_value_counts.sort_index()

# Set display options to show all rows
pd.set_option('display.max_rows', None)

# Print the sorted counts
print(SN_sorted_value_counts)

Snatch (lbs)
44.0924                  8
45.0                    11
47.0                     2
48.0                     1
48.50164                 4
50.0                    18
50.70625999999999        2
52.0                     1
52.91087999999999        1
55.0                    10
55.1155                 10
56.0                     1
57.0                     1
59.52474                 1
60.0                    12
61.72935999999999        3
62.0                     3
63.0                     1
63.93397999999999        4
64.0                     1
65.0                    34
66.0                     1
66.1386                 38
67.0                     2
68.0                     2
68.34321999999999        4
69.0                     1
70.0                    29
70.54784                 5
71.0                     2
72.0                     1
72.75246                 4
73.0                     2
74.0                     1
74.95707999999999        6
75.0                    53
76.0           

In [185]:
# Replace values greater than 550 with NaN
Men_Clean_Me.loc[(Men_Clean_Me['Fight Gone Bad'] > 450), 'Fight Gone Bad'] = pd.NaT

In [186]:
# Count the occurrences of each unique value in the 'col1' column
FGB_value_counts = Men_Clean_Me['Fight Gone Bad'].value_counts()

# Sort the value counts in descending order
FGB_sorted_value_counts = FGB_value_counts.sort_index()

# Set display options to show all rows
pd.set_option('display.max_rows', None)

# Print the sorted counts
print(FGB_sorted_value_counts)

Fight Gone Bad
1.0       38
2.0        3
3.0        5
4.0        1
5.0        1
6.0        1
7.0        4
8.0        1
10.0       2
11.0       1
12.0       1
15.0       4
20.0       3
21.0       1
25.0       2
28.0       1
30.0       2
38.0       2
40.0       1
41.0       1
42.0       1
45.0       1
50.0       2
55.0       1
87.0       1
92.0       1
98.0       1
100.0      5
101.0      1
104.0      1
105.0      1
116.0      1
120.0      1
121.0      1
123.0      1
124.0      1
125.0      1
133.0      1
134.0      1
136.0      2
139.0      1
141.0      1
146.0      1
148.0      1
150.0      8
153.0      2
154.0      3
155.0      3
156.0      3
157.0      2
158.0      2
160.0      4
161.0      3
163.0      2
164.0      3
165.0      3
166.0      2
167.0      3
168.0      1
169.0      3
170.0      3
171.0      1
173.0      2
174.0      5
175.0      4
176.0      4
177.0      3
178.0      5
179.0      2
180.0      7
181.0      4
182.0      5
183.0      8
184.0      6
185.0      6
186.0     

In [187]:
# Count the occurrences of each unique value in the 'col1' column
F50_value_counts = Men_Clean_Me['Filthy 50'].value_counts()

# Sort the value counts in descending order
F50_sorted_value_counts = F50_value_counts.sort_index()

# Set display options to show all rows
pd.set_option('display.max_rows', None)

# Print the sorted counts
print(F50_sorted_value_counts)

Filthy 50
0:01         5
0:08         1
0:10         2
0:58         1
10000:39     1
1000:00      1
100:09       1
100:39       2
10:10        1
10:15        1
10:41        1
11:01        1
11:36        1
12:00        2
12:04        1
12:31        1
12:45        1
12:56        1
130:19       1
13:12        1
13:15        1
13:31        1
13:33        1
13:34        1
13:35        1
13:38        1
13:39        1
13:40        1
13:58        1
14:00        1
14:04        1
14:05        1
14:11        1
14:21        1
14:23        1
14:25        1
14:30        2
14:35        1
14:40        1
14:41        1
14:42        2
14:44        1
14:45        1
14:48        1
14:50        3
14:51        1
14:52        1
14:54        1
14:55        1
14:56        1
14:57        1
14:58        1
14:59        2
15:00        1
15:05        3
15:07        2
15:08        1
15:13        1
15:17        2
15:18        1
15:19        1
15:20        3
15:22        2
15:23        2
15:25        1
15:26        1


In [188]:
#Output Crossfit to a CSV file
Men_Clean_Me.to_csv('Pre_DT_Crossfit.csv', index=False)

In [197]:
Pre_DT = pd.read_csv('/Users/cdmoseley/dai_files/Capstone/Galvanize_Capstone_Crossfit/Data/Pre_DT_Crossfit.csv')
Pre_DT.head()

/var/folders/xm/kc32pwjj5fj5xqf43yb4dz340000gn/T/ipykernel_44180/3556571759.py:1: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  Pre_DT = pd.read_csv('/Users/cdmoseley/dai_files/Capstone/Galvanize_Capstone_Crossfit/Data/Pre_DT_Crossfit.csv')


,Athlete ID,Gender,Competitor Name,Country Name,Region Name,Affiliate Name,Height (in),Age,Weight (lbs),Overall Rank,24.1 Rank,24.2 Rank,24.3 Rank,2024 Rank,2023 Rank,2022 Rank,2021 Rank,2020 Rank,2019 Rank,2018 Rank,2017 Rank,2016 Rank,2015 Rank,2014 Rank,2013 Rank,2012 Rank,2011 Rank,Back Squat (lbs),Deadlift (lbs),Clean and Jerk (lbs),Snatch (lbs),Fight Gone Bad,Filthy 50,Fran,Grace,Helen,L1 Benchmark,Run 5k,Sprint 400m
0,10942,M,Everett Sloan,Canada,North America East,CrossFit Bytown,67.0,45.0,200.0,68985.0,51383.0,84516.0,80225.0,68985.0,62537.0,66.0,NaN,126461.0,64200.0,19449.0,20303.0,8704.0,4733.0,3491.0,1372.0,3243.0,5051.0,425.0,585.0,285.0,225.0,NaN,NaN,2:32,1:47,7:13,NaN,NaN,NaN
1,10926,M,Kevin Wood,Canada,North America East,CrossFit Moncton,75.0,43.0,196.0,35977.0,74333.0,17860.0,27790.0,35977.0,30964.0,29723.0,21170.0,18926.0,19944.0,16438.0,12456.0,11723.0,10968.0,9164.0,4686.0,3880.0,NaN,365.0,405.0,275.0,190.0,429.0,19:47,3:16,3:13,8:16,NaN,19:38,1:00
2,10976,M,John Lazaro,United States,North America East,CrossFit Bridgewater,69.0,46.0,180.0,164180.0,156669.0,154294.0,142271.0,164180.0,143566.0,119744.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14806.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,10977,M,Matthew Haynes,United States,North America West,CrossFit Westwood,71.0,31.0,180.0,165637.0,160697.0,154294.0,142271.0,165637.0,157742.0,87420.0,56772.0,50805.0,43038.0,24249.0,84288.0,NaN,NaN,48836.0,407.0,661.0,NaN,400.0,485.0,300.0,255.0,NaN,NaN,2:00,NaN,NaN,NaN,NaN,NaN
4,10966,M,Nick DiMatteo,United States,North America East,CrossFit OTG,68.0,48.0,165.0,60027.0,47496.0,89266.0,54110.0,60027.0,49886.0,55120.0,37826.0,22196.0,34774.0,45660.0,55386.0,46349.0,29146.0,19472.0,12970.0,5729.0,1996.0,315.0,400.0,225.0,165.0,NaN,NaN,3:41,NaN,9:01,NaN,21:55,NaN
